## 0. Import Library

In [ ]:
! pip install category_encoders

In [2]:
# data analysis related library
import pandas as pd
import numpy as np

# some visualization related library
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from matplotlib.colors import LogNorm
import seaborn as sns

# category_encoders and warning related library
from warnings import simplefilter
simplefilter(action='ignore')
import category_encoders

# sklearn related ML library
from sklearn import metrics
from sklearn.metrics import roc_curve, RocCurveDisplay
from sklearn.metrics import mean_absolute_error
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# other useful libray
import missingno as msno
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from pylab import subplots_adjust
import copy
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# data analysis related library
import pandas as pd
import numpy as np

# some visualization related library
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from matplotlib.colors import LogNorm
import seaborn as sns

# category_encoders and warning related library
from warnings import simplefilter
simplefilter(action='ignore')
import category_encoders

# sklearn related ML library
from sklearn import metrics
from sklearn.metrics import roc_curve, RocCurveDisplay
from sklearn.metrics import mean_absolute_error
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# other useful libray
import missingno as msno
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from pylab import subplots_adjust
import copy
import torch
import torch.nn as nn
import torch.optim as optim

## 1. Read Data

In [4]:
health_data = pd.read_csv("framingham.csv")

In [ ]:
health_data.info()

In [ ]:
health_data.shape

In [ ]:
health_data.head()

In [ ]:
health_data.describe()

In [ ]:
health_data.isna().sum().sum()

In [ ]:
health_data.isnull().sum().sort_values(ascending=False)

In [ ]:
(health_data.isnull().sum() / health_data.isnull().count()).sort_values(ascending=False)

In [ ]:
msno.dendrogram(health_data)
plt.show()

In [ ]:
msno.heatmap(health_data)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
msno.matrix(health_data, color=(0.6, 0.2, 0.8))  
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

missing_counts = health_data.isnull().sum()

num_columns = len(missing_counts)
half = num_columns // 2
cols_part1 = missing_counts[:half]
cols_part2 = missing_counts[half:]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))  

axes[0].barh(cols_part1.index, cols_part1.values, color=(255/255, 151/255, 0/255))
#axes[0].set_title("Missing Values")
axes[0].set_xlabel("Count")

axes[1].barh(cols_part2.index, cols_part2.values, color=(255/255, 151/255, 0/255))
#axes[1].set_title("Missing Values")
axes[1].set_xlabel("Count")

plt.tight_layout()
plt.show()


In [ ]:
msno.bar(health_data.sample(1000), color=(255/255,151/225,0/255))
plt.show()

## 2. Data Cleaning, Analysis and Featrue Selection

### 2.1 Fill Missing Value

In [ ]:
health_data.TenYearCHD.value_counts()

In [ ]:
# search the numerical features
numerical_features = health_data.columns[health_data.dtypes != object]
list(numerical_features)

In [ ]:
num_features = len(health_data.columns)  
num_rows = 2  
num_cols = (num_features + num_rows - 1) // num_rows  

fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 4))

axes = axes.flatten()

for i, col in enumerate(health_data.columns):
    sns.histplot(x=col, data=health_data, ax=axes[i], color=(159/255, 57/255, 222/255))

    
    axes[i].set_xlabel(col, fontsize=16) 
    axes[i].set_ylabel("Count", fontsize=16)  

plt.tight_layout()
plt.show()




In [21]:
health_data_filled_with_mean = pd.DataFrame()
for col in health_data.columns:
    if health_data[col].dtype == 'float64':
        health_data_filled_with_mean[col] = health_data[col].fillna(health_data[col].mean())
    elif set(health_data[col].dropna().unique()).issubset({0, 1}):
        health_data_filled_with_mean[col] = health_data[col].fillna(health_data[col].median())
    else:
        health_data_filled_with_mean[col] = health_data[col]



In [ ]:
num_features = len(health_data_filled_with_mean.columns)  
num_rows = 2  
num_cols = (num_features + num_rows - 1) // num_rows  

fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 4))

axes = axes.flatten()

for i, col in enumerate(health_data_filled_with_mean.columns):
    sns.histplot(x=col, data=health_data_filled_with_mean, ax=axes[i], color=(159/255, 57/255, 222/255))

    axes[i].set_xlabel(col, fontsize=16)
    axes[i].set_ylabel("Count", fontsize=16)

    axes[i].tick_params(axis='x', labelsize=14)
    axes[i].tick_params(axis='y', labelsize=14)

plt.tight_layout()
plt.show()

In [ ]:
msno.matrix(health_data_filled_with_mean,color=(47/255,127/255,255/255))
msno.matrix(health_data_filled_with_mean, color=(0.6, 0.2, 0.8))  
plt.show()

In [ ]:
health_data_filled_with_mean.isnull().sum().sort_values()

In [ ]:
health_data_filled_with_mean.info()

In [26]:
health_data_filled_with_mean['BPMeds'] = health_data_filled_with_mean['BPMeds'].astype('int64')

In [ ]:
health_data_filled_with_mean.info()

### 2.2 Handle outliers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update({
    'font.size': 18,            
    'font.weight': 'normal',   
    'axes.labelweight': 'normal',
    'axes.titleweight': 'normal'
})

fig = plt.figure(figsize=(35, 40))
fig.set_facecolor('white')  

index = 1
target_col = 'TenYearCHD'

box_color = (0/255, 100/255, 200/255)  

for k in health_data_filled_with_mean:
    if health_data_filled_with_mean[k].dtype == 'float64':
        ax = plt.subplot(len(numerical_features), 2, index)
        sns.boxplot(
            x=health_data_filled_with_mean[target_col].astype(str),  
            y=health_data_filled_with_mean[k],
            color=box_color  
        )

        plt.title(f'Box Plot of {k} by {target_col}', fontsize=12, fontweight='normal')
        plt.xlabel('TenYearCHD', fontsize=12, fontweight='normal')
        plt.ylabel(k, fontsize=12, fontweight='normal')

        ax.yaxis.grid(True, linestyle='--', alpha=0.7)

        for spine in ax.spines.values():
            spine.set_edgecolor('darkblue')  
            spine.set_linewidth(2)  
            spine.set_linestyle('-')  

        ax.spines['top'].set_visible(False)  
        ax.spines['right'].set_visible(False)  
        ax.spines['left'].set_color('black')  
        ax.spines['bottom'].set_color('black')  

        index += 1

plt.tight_layout()
plt.show()


### 2.2.1 Handle outliers for TenYearCHD = 0

In [31]:
group_0 = health_data_filled_with_mean[health_data_filled_with_mean['TenYearCHD'] == 0]

In [ ]:
fig = plt.figure(figsize=(35,20))
fig.set_facecolor((235/255,235/255,235/255))
index = 1
for k in group_0:
    if group_0[k].dtype=='float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_0[k], color=(0/255,219/255,197/255))
        index += 1
plt.tight_layout()
plt.show()

In [33]:
for col in group_0.columns:
    if col != 'TenYearCHD' and group_0[col].dtype == 'float64':
        Q1 = group_0[col].quantile(0.25)
        Q3 = group_0[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        group_0[col] = np.where(group_0[col] < lower_bound, lower_bound, group_0[col])
        group_0[col] = np.where(group_0[col] > upper_bound, upper_bound, group_0[col])

In [ ]:
fig = plt.figure(figsize=(35,20))
fig.set_facecolor((235/255,235/255,235/255))
index = 1
for k in group_0:
    if group_0[k].dtype=='float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_0[k], color=(0/255,219/255,197/255))
        index += 1
plt.tight_layout()
plt.show()

### 2.2.2 For TenYearCHD = 1

In [35]:
group_1 = health_data_filled_with_mean[health_data_filled_with_mean['TenYearCHD'] == 1]

In [ ]:
fig = plt.figure(figsize=(35,20))
fig.set_facecolor((235/255,235/255,235/255))
index = 1
for k in group_1:
    if group_1[k].dtype=='float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_1[k], color=(0/255,219/255,197/255))
        index += 1
plt.tight_layout()
plt.show()

In [37]:
for col in group_1.columns:
    if col != 'TenYearCHD' and group_1[col].dtype == 'float64':
        Q1 = group_1[col].quantile(0.25)
        Q3 = group_1[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        group_1[col] = np.where(group_1[col] < lower_bound, lower_bound, group_1[col])
        group_1[col] = np.where(group_1[col] > upper_bound, upper_bound, group_1[col])

In [ ]:
fig = plt.figure(figsize=(35,20))
fig.set_facecolor((235/255,235/255,235/255))
index = 1
for k in group_1:
    if group_1[k].dtype=='float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_1[k], color=(0/255,219/255,197/255))
        index += 1
plt.tight_layout()
plt.show()

As the number of 1 is small, we keep all the outliers to let the model learn all the features

In [39]:
health_data_filled_with_mean = pd.concat([group_0, group_1])

### 2.2.3 Final box plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update({
    'font.size': 18,            
    'font.weight': 'normal',   
    'axes.labelweight': 'normal',
    'axes.titleweight': 'normal'
})

fig = plt.figure(figsize=(35, 40))
fig.set_facecolor('white')  

index = 1
target_col = 'TenYearCHD'

box_color = (0/255, 100/255, 200/255)  

for k in health_data_filled_with_mean:
    if health_data_filled_with_mean[k].dtype == 'float64':
        ax = plt.subplot(len(numerical_features), 2, index)

        sns.boxplot(
            x=health_data_filled_with_mean[target_col].astype(str), 
            y=health_data_filled_with_mean[k],
            color=box_color 
        )


        plt.title(f'Box Plot of {k} by {target_col}', fontsize=12, fontweight='normal')
        plt.xlabel('TenYearCHD', fontsize=12, fontweight='normal')
        plt.ylabel(k, fontsize=12, fontweight='normal')


        ax.yaxis.grid(True, linestyle='--', alpha=0.7)


        for spine in ax.spines.values():
            spine.set_edgecolor('darkblue')  
            spine.set_linewidth(2)  
            spine.set_linestyle('-')  

        ax.spines['top'].set_visible(False)  
        ax.spines['right'].set_visible(False)  
        ax.spines['left'].set_color('black')  
        ax.spines['bottom'].set_color('black')  

        index += 1

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(35, 40))
fig.set_facecolor((235/255, 235/255, 235/255))

index = 1
target_col = 'TenYearCHD'

for k in health_data_filled_with_mean:
    if health_data_filled_with_mean[k].dtype == 'float64':
        plt.subplot(len(numerical_features), 2, index)
        sns.boxplot(
            x=health_data_filled_with_mean[target_col].astype(str), 
            y=health_data_filled_with_mean[k],
            palette={'0': (0/255, 219/255, 197/255), '1': (255/255, 99/255, 132/255)}
        )
        plt.title(f'Box Plot of {k} by {target_col}')
        plt.xlabel('TenYearCHD')
        plt.ylabel(k)
        index += 1

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(35, 40))
fig.set_facecolor((235/255, 235/255, 235/255))

index = 1
target_col = 'TenYearCHD'

for k in health_data_filled_with_mean:
    if health_data_filled_with_mean[k].dtype == 'float64':
        plt.subplot(len(numerical_features), 2, index)
        sns.boxplot(
            x=health_data_filled_with_mean[target_col].astype(str),
            y=health_data_filled_with_mean[k],
            palette={'0': (0/255, 219/255, 197/255), '1': (255/255, 99/255, 132/255)}
        )
        plt.title(f'Box Plot of {k} by {target_col}', fontsize=20)  
        plt.xlabel('TenYearCHD', fontsize=18)  
        plt.ylabel(k, fontsize=18)  
        plt.xticks(fontsize=16)  
        plt.yticks(fontsize=16)  
        index += 1

plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(35, 20))
fig.set_facecolor((235/255, 235/255, 235/255))
index = 1

for k in group_0:
    if group_0[k].dtype == 'float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_0[k], color=(0/255, 219/255, 197/255))
        plt.title(f'Box Plot of {k} (TenYearCHD=0)', fontsize=18)
        plt.xlabel(k, fontsize=16)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        index += 1

plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(35, 20))
fig.set_facecolor((235/255, 235/255, 235/255))
index = 1

for k in group_1:
    if group_1[k].dtype == 'float64':
        plt.subplot(4, 6, index)
        sns.boxplot(group_1[k], color=(255/255, 99/255, 132/255))
        plt.title(f'Box Plot of {k} (TenYearCHD=1)', fontsize=18)
        plt.xlabel(k, fontsize=16)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        index += 1

plt.tight_layout()
plt.show()


## 2.4 Visualization and Drop highly-correlated features

In [ ]:
health_data_filled_with_mean.info()

In [ ]:
corr_matrix = health_data_filled_with_mean.corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(12, 11))
sns.heatmap(corr_matrix, mask=mask, cmap="YlGnBu", annot=True, fmt='.2f')
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
features_correlation = abs(corr_matrix['TenYearCHD']).sort_values(ascending=False)

size = [
    features_correlation.iloc[1], features_correlation.iloc[2],
    features_correlation.iloc[3], features_correlation.iloc[4],
    features_correlation.iloc[5], features_correlation.iloc[6],
    features_correlation.iloc[7], features_correlation.iloc[8],
    features_correlation.iloc[9], features_correlation.iloc[10],
    features_correlation.iloc[11], features_correlation.iloc[12],
    features_correlation.iloc[13], features_correlation.iloc[14],
    features_correlation.iloc[15],
]

labels = [
    features_correlation.index[1], features_correlation.index[2],
    features_correlation.index[3], features_correlation.index[4],
    features_correlation.index[5], features_correlation.index[6],
    features_correlation.index[7], features_correlation.index[8],
    features_correlation.index[9], features_correlation.index[10],
    features_correlation.index[11], features_correlation.index[12],
    features_correlation.index[13], features_correlation.index[14],
    features_correlation.index[15],
]

explode = [0.1] * 5 + [0] * (len(size) - 5)

plt.figure(figsize=(8, 8))
plt.pie(size, labels=labels, explode=explode, autopct='%1.2f%%')
plt.title('Correlation with TenYearCHD (Top 5 Highlighted)', fontsize=16, fontweight='bold')
plt.show()



CurrentSmoker has a strong correlation with cigsPerDay (approximately 0.77), indicating a significant linear relationship between the two features. Since cigsPerDay is a float feature that captures more granular information about smoking intensity (e.g., the number of cigarettes smoked per day), we propose retaining cigsPerDay as the primary smoking-related feature and removing CurrentSmoker to reduce feature redundancy.

Also, we think that diaBP and sysBP can presents better than int64 feature prevalentHyp.

In [45]:
health_data_filled_with_mean_cleaned = health_data_filled_with_mean.drop('currentSmoker', axis=1)
health_data_filled_with_mean_cleaned = health_data_filled_with_mean_cleaned.drop('prevalentHyp', axis=1)

In [ ]:
corr_matrix = health_data_filled_with_mean_cleaned.corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(12, 11))
sns.heatmap(corr_matrix, mask=mask, cmap="YlGnBu", annot=True, fmt='.2f')
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
health_data_filled_with_mean_cleaned.info()

### 2.5 Excute Balance and Scale

In [68]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.patches import FancyArrowPatch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd

X = health_data_filled_with_mean_cleaned.drop(columns=['TenYearCHD'])
y = health_data_filled_with_mean_cleaned['TenYearCHD']

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

smote = SMOTE(random_state=42)
x_train_upsampled, y_train_upsampled = smote.fit_resample(x_train, y_train)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_upsampled)
x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train_upsampled.columns)
x_test_scaled = scaler.transform(x_test)
x_train_scaled = x_train_scaled.drop(columns=['gender', 'education'], errors='ignore')
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns)  
x_test_scaled = x_test_scaled.drop(columns=['gender', 'education'], errors='ignore')

# tsne pca

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from matplotlib.patches import FancyArrowPatch
from matplotlib.gridspec import GridSpec
import random

tsne = TSNE(n_components=2, random_state=42, perplexity=50, learning_rate='auto', n_iter=1000)
tsne_results = tsne.fit_transform(x_train_scaled)

fig, axes = plt.subplots(1, 2, figsize=(16, 6)) 

sns.scatterplot(ax=axes[0], x=tsne_results[:, 0], y=tsne_results[:, 1], hue=y_train_upsampled, 
                palette="coolwarm", alpha=0.7, s=30)
axes[0].set_xlabel("t-SNE Component 1", fontsize=12)
axes[0].set_ylabel("t-SNE Component 2", fontsize=12)
axes[0].legend(title="TenYearCHD", fontsize=10, title_fontsize=11, loc='upper left',
                        markerscale=0.7, bbox_to_anchor=(0, 1))
#axes[0].set_title("t-SNE Visualization of CHD Data")

pca = PCA(n_components=2)
pca_2d = pca.fit_transform(x_train_scaled)
pca_components = PCA(n_components=2).fit(x_train_scaled)

cmap = plt.cm.plasma  

def draw_arrow(ax, start, end, color):
    arrow = FancyArrowPatch(posA=start, posB=end, arrowstyle='->',
                            color=color, linewidth=1.5, mutation_scale=15)
    ax.add_artist(arrow)

arrow_lengths = np.sqrt(np.sum(pca_components.components_**2, axis=0))
max_arrow_length = np.max(arrow_lengths)
scaling_factor = 0.9 / max_arrow_length  

contributions = np.sqrt(np.sum(pca_components.components_**2, axis=0))
contribution_normalized = 20 * (contributions - np.min(contributions)) / (np.max(contributions) - np.min(contributions))

jitter = lambda: random.uniform(-0.05, 0.05)  


for i, feature in enumerate(x_train_scaled.columns):
    contribution = contribution_normalized[i]
    scaled_x = pca_components.components_[0, i] * scaling_factor
    scaled_y = pca_components.components_[1, i] * scaling_factor
    color = cmap(contribution / 25)  
    draw_arrow(axes[1], (0, 0), (scaled_x, scaled_y), color=color)
    
    offset_x = -0.1 * np.sign(scaled_x) if abs(scaled_x) < 0.3 else -0.08 * np.sign(scaled_x)
    offset_y = -0.1 * np.sign(scaled_y) if abs(scaled_y) < 0.3 else -0.08 * np.sign(scaled_y)
    
    if abs(scaled_x) < 0.2 and abs(scaled_y) < 0.2:
        offset_x *= 2  
        offset_y *= 2
    
    offset_x += jitter()
    offset_y += jitter()

    axes[1].text(scaled_x * 1.15 + offset_x, scaled_y * 1.15 + offset_y, feature,
                 color=color, fontsize=12, ha='center', va='center', fontweight='medium', zorder=10)

    #offset_x = -0.08 * np.sign(scaled_x)
    #offset_y = -0.08 * np.sign(scaled_y)
    #axes[1].text(scaled_x * 1.15 + offset_x, scaled_y * 1.15 + offset_y, feature,
                 #color=color, fontsize=12, ha='center', va='center', fontweight='medium', zorder=10)



axes[1].set_xlabel(f'Dim1 ({pca.explained_variance_ratio_[0] * 100:.1f}%)', fontsize=12)
axes[1].set_ylabel(f'Dim2 ({pca.explained_variance_ratio_[1] * 100:.1f}%)', fontsize=12)
#axes[1].set_title("PCA Biplot of Feature Contributions")
axes[1].grid(True, linestyle='--', linewidth=0.6, alpha=0.7)
axes[1].axhline(0, color='black', linestyle='--', linewidth=0.6)
axes[1].axvline(0, color='black', linestyle='--', linewidth=0.6)
axes[1].add_patch(plt.Circle((0, 0), 1, color='black', fill=False, linestyle='--', linewidth=0.6))
axes[1].set_aspect('equal', adjustable='datalim')

plt.grid(True, color='lightgray', linestyle='-', linewidth=0.5)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=20))
sm.set_array([])
cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical', fraction=0.03, pad=0.04)
cbar.set_label('contrib', fontsize=9)
cbar.ax.tick_params(labelsize=10) 

axes[0].tick_params(axis='both', labelsize=9)  
axes[1].tick_params(axis='both', labelsize=9)  

axes[0].set_xlim(-60, 60)
axes[0].set_ylim(-60, 60)
axes[1].set_xlim(-1, 1)
axes[1].set_ylim(-1, 1)

plt.tight_layout()
plt.show()
